In [ ]:
%pip install llama-index-vector-stores-lancedb
%pip install llama-index-multi-modal-llms-openai

In [ ]:
#%pip install llama-index-multi-modal-llms-openai
#%pip install llama-index-vector-stores-lancedb
%pip install llama-index-embeddings-clip

In [ ]:
%pip install llama_index ftfy regex tqdm
%pip install -U openai-whisper
%pip install git+https://github.com/openai/CLIP.git
%pip install torch torchvision
%pip install matplotlib scikit-image
%pip install lancedb
%pip install moviepy
%pip install pytube
%pip install pydub
%pip install SpeechRecognition
%pip install ffmpeg-python
%pip install soundfile

In [ ]:
from moviepy.editor import VideoFileClip
from pathlib import Path
import speech_recognition as sr
from pytube import YouTube
from pprint import pprint

In [ ]:
from google.colab import userdata
OPENAI_API_TOKEN=userdata.get('OPENAI_API_KEY')

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_TOKEN

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

In [ ]:
import os
os.getcwd()

In [ ]:
video_url="https://youtu.be/3dhcmeOTZ_Q"
output_video_path = "/content/video_data/"
output_folder = "/content/mixed_data/"
output_audio_path = "/content/mixed_data/output_audio.wav"

In [ ]:
filepath=output_video_path + "input_vid.mp4"

In [ ]:
filepath

In [ ]:
#filepath = output_video_path + "input_vid.mp4"
#Path(output_folder).mkdir(parents=True, exist_ok=True)

In [ ]:
#Path(filepath).mkdir(parents=True,exist_ok=True)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_images(images_path):
  images_shown = 0
  plt.figure(figsize=(16, 9))
  for img_path in images_path:
        if os.path.isfile(img_path):
            image = Image.open(img_path)

            plt.subplot(2, 3, images_shown + 1)
            plt.imshow(image)
            plt.xticks([])
            plt.yticks([])

            images_shown += 1
            if images_shown >= 5:
                break


In [ ]:
from pytube import YouTube
def download_video(url,output_path):
  yt = YouTube(url)
  metadata = {"Author": yt.author, "Title": yt.title, "Views": yt.views}
  yt.streams.get_highest_resolution().download(
        output_path=output_path, filename="input_vid.mp4"
    )
  return metadata

In [ ]:
from moviepy.editor import VideoFileClip
def video_to_images(video_path,output_flder):
  clip=VideoFileClip(video_path)
  clip.write_images_sequence(
      os.path.join(output_folder,"frame%04d.png"),fps=0.2
  )

In [ ]:
def video_to_audio(video_path,output_audio_path):
  clip=VideoFileClip(video_path)
  audio=clip.audio
  audio.write_audiofile(output_audio_path)


In [ ]:
def audio_to_text(audio_path):
  recognizer=sr.Recognizer()
  audio=sr.AudioFile(audio_path)

  with audio as source:
    audio_data=recognizer.record(source)

    try:

      #recognize the speech
      text = recognizer.recognize_whisper(audio_data)

    except sr.UnknownValueError:
      print("Speech recognition could not understand the audio.")
  return text



In [ ]:
video_url

In [ ]:
output_video_path

In [ ]:
metadata_vid = download_video(video_url, output_video_path)

In [ ]:
metadata_vid

In [ ]:
filepath

In [ ]:
output_folder

In [ ]:
!mkdir mixed_data

In [ ]:
filepath

In [ ]:
output_folder

In [ ]:
video_to_images(filepath,output_folder)

In [ ]:
filepath

In [ ]:
output_audio_path

In [ ]:
video_to_audio(filepath,output_audio_path)

In [ ]:
output_audio_path

In [ ]:
text_data=audio_to_text(output_audio_path)

In [ ]:
text_data

In [ ]:
with open(output_folder + "output_text.txt", "w") as file:
        file.write(text_data)
print("Text data saved to file")
file.close()




In [ ]:
os.remove(output_audio_path)
print("Audio file removed")

In [ ]:
# embedding lance db
# llm

In [ ]:
!pip install llama-index-readers-file

In [ ]:
from llama_index.core.indices import MultiModalVectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.core import StorageContext

In [ ]:
from llama_index.vector_stores.lancedb import LanceDBVectorStore

In [ ]:
text_store=LanceDBVectorStore(uri="lancedb",table_name="text_collection")
image_store=LanceDBVectorStore(uri="lancedb",table_name="image_collection")

In [ ]:
storage_context=StorageContext.from_defaults(vector_store=text_store,image_store=image_store)

In [ ]:
output_folder

In [ ]:
documents=SimpleDirectoryReader(output_folder).load_data()

In [ ]:
documents

In [ ]:
index = MultiModalVectorStoreIndex.from_documents(documents,storage_context=storage_context)

In [ ]:
retriever_engine=index.as_retriever(imilarity_top_k=5, image_similarity_top_k=5)

# RAG

In [ ]:
import json
metadata_str=json.dumps(metadata_vid)

In [ ]:
qa_tmpl_str=(
    "Given the provided information, including relevant images and retrieved context from the video, \
    accurately and precisely answer the query without any additional prior knowledge.\n"

    "---------------------\n"
    "Context: {context_str}\n"
    "Metadata for video: {metadata_str} \n"

    "---------------------\n"
    "Query: {query_str}\n"
    "Answer: "
)

In [ ]:
qa_tmpl_str2=(
    "Given the provided information, including relevant images and retrieved context from the video, \
    accurately and precisely answer the query without any additional prior knowledge.\n"

    "---------------------\n"
    "Metadata for video: {metadata_str} \n"

    "---------------------\n"
    "Query: {query_str}\n"
    "Answer: "
)

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node
from llama_index.core.schema import ImageNode

In [ ]:
query_str="what is a O'Reilly platform.?"

In [ ]:
retrieval_results=retriever_engine.retrieve(query_str)

In [ ]:
retrieval_results

In [ ]:
retrieved_image=[]
retrieved_text=[]

In [ ]:
for res_node in retrieval_results:
  if isinstance(res_node.node,ImageNode):
    retrieved_image.append(res_node.node.metadata["file_path"])
  else:
     display_source_node(res_node, source_length=200)
     retrieved_text.append(res_node.text)




In [ ]:
retrieved_image

In [ ]:
retrieved_text

In [ ]:
def retrieve(retriever_engine, query_str):
    retrieval_results = retriever_engine.retrieve(query_str)

    retrieved_image = []
    retrieved_text = []
    for res_node in retrieval_results:
        if isinstance(res_node.node, ImageNode):
            retrieved_image.append(res_node.node.metadata["file_path"])
        else:
            display_source_node(res_node, source_length=200)
            retrieved_text.append(res_node.text)

    return retrieved_image, retrieved_text

In [ ]:
img, txt = retrieve(retriever_engine=retriever_engine, query_str=query_str)

In [ ]:
output_folder

In [ ]:
img

In [ ]:
image_documents = SimpleDirectoryReader(input_dir=output_folder, input_files=img).load_data()

In [ ]:
image_documents2 = SimpleDirectoryReader(input_files=img).load_data()

In [ ]:
context_str = "".join(txt)

In [ ]:
context_str

In [ ]:
plot_images(img)

In [ ]:
qa_tmpl_str2

In [ ]:
from llama_index.multi_modal_llms.openai import OpenAIMultiModal

In [ ]:
openai_mm_llm = OpenAIMultiModal(model="gpt-4-vision-preview", api_key=OPENAI_API_TOKEN, max_new_tokens=1500)

In [ ]:
result=openai_mm_llm.complete(
    prompt=qa_tmpl_str2.format(
        query_str=query_str,metadata_str=metadata_str
    ),
    image_documents=image_documents2,
)

In [ ]:
pprint(result.text)

In [ ]:
result2=openai_mm_llm.complete(
    prompt="what is O'Reilly?",
    image_documents=image_documents2
)
print(result2.text)

In [ ]:
result3=openai_mm_llm.complete(
    prompt="just tell me what you are seeing over the pecific ocean?",
    image_documents=image_documents2
)
pprint(result3.text)

#real time system

#sattalites

#just tell me what you are seeing over the pecific ocean?

In [ ]:
realtime idea:

